In [ ]:
import ee
import shutil
import os

In [ ]:
ee.Authenticate(force=True)
ee.Initialize()

In [ ]:
def lambert_azimuthal_equal_area_wkt_generation(centroid_lat, centroid_lon, file_path):
    projection = """
    PROJCS["Lambert Azimuthal Equal Area",
        GEOGCS["GCS_WGS_1984",
            DATUM["WGS_1984",
                SPHEROID["WGS_84",6378137,298.257223563]],
            PRIMEM["Greenwich",0],
            UNIT["Degree",0.0174532925199433]],
        PROJECTION["Lambert_Azimuthal_Equal_Area"],
        PARAMETER["latitude_of_center", {centroid_lat}],
        PARAMETER["longitude_of_center", {centroid_lon}],
        PARAMETER["false_easting", 0],
        PARAMETER["false_northing", 0],
        UNIT["Meter",1]]
    """
    modified_projection = projection.format(centroid_lat=centroid_lat, centroid_lon=centroid_lon)
    
    # Saving the modified projection to the specified file path
    with open(file_path, 'w') as file:
        file.write(modified_projection)
    
    print(f"Projection saved to {file_path}")

def lambert_conformal_conic(parallel_lat_1, parallel_lat_2, central_meridian, latitude_of_origin, file_path, false_easting=0, false_northing=0,  gcs_scale=0.00026949458523585647, pcs_scale=30):
    projection = """
    PROJCS["Custom LCC Projection",
        GEOGCS["GCS_WGS_1984",
            DATUM["WGS_1984",
                SPHEROID["WGS_84",6378137,298.257223563]],
            PRIMEM["Greenwich",0],
            UNIT["Degree",{gcs_scale}]],
        PROJECTION["Lambert_Conformal_Conic"],
        PARAMETER["Standard_Parallel_1", {parallel_lat_1}],
        PARAMETER["Standard_Parallel_2", {parallel_lat_2}],
        PARAMETER["Central_Meridian", {central_meridian}],
        PARAMETER["Latitude_Of_Origin", {latitude_of_origin}],
        PARAMETER["False_Easting", {false_easting}],
        PARAMETER["False_Northing", {false_northing}],
        UNIT["Meter",{pcs_scale}]]
    """
    modified_projection = projection.format(parallel_lat_1=parallel_lat_1, parallel_lat_2=parallel_lat_2, central_meridian=central_meridian, latitude_of_origin=latitude_of_origin, false_easting=false_easting, false_northing=false_northing)
    
    # Saving the modified projection to the specified file path
    with open(file_path, 'w') as file:
        file.write(modified_projection)
    
    print(f"Projection saved to {file_path}")
    

In [ ]:
hybas_lev02  = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100")
hybas_lev02_size = hybas_lev02.size().getInfo()
print(hybas_lev02_size)
hybas_lev02_list = hybas_lev02.toList(hybas_lev02_size)

In [ ]:
MODE = 'Lambert_Azimuthal_Equal_Area'
for i in range(hybas_lev02_size):
    hybas_lev02_feature = ee.Feature(hybas_lev02_list.get(i))
    hybas_lev02_id = hybas_lev02_feature.get('HYBAS_ID').getInfo()
    hybas_lev02_centroid = hybas_lev02_feature.centroid()
    hybas_lev02_centroid_coords = hybas_lev02_centroid.geometry().coordinates().getInfo()
    print(hybas_lev02_centroid_coords)
    hybas_lev02_centroid_lat = hybas_lev02_centroid_coords[1]
    hybas_lev02_centroid_lon = hybas_lev02_centroid_coords[0]
    file_path = f'./projection_wkt/{MODE}/{hybas_lev02_id}.wkt'
    output_dir = os.path.dirname(file_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    lambert_azimuthal_equal_area_wkt_generation(hybas_lev02_centroid_lat, hybas_lev02_centroid_lon, file_path)
    
    print(f"Hybas Lev02 ID: {hybas_lev02_id} projection WKT file saved to {file_path}")

In [ ]:
MODE = 'Lambert_Conformal_Conic'
GCS_SCALE = 0.00026949458523585647
PCS_SCALE = 30
for i in range(hybas_lev02_size):
    hybas_lev02_feature = ee.Feature(hybas_lev02_list.get(i))
    hybas_lev02_id = hybas_lev02_feature.get('HYBAS_ID').getInfo()
    hybas_lev02_bounds = hybas_lev02_feature.bounds()
    hybas_lev02_bounds_coords = hybas_lev02_bounds.geometry().coordinates().getInfo()[0]
    print(hybas_lev02_bounds_coords)
    lat_lower_quarters = hybas_lev02_bounds_coords[0][1] + (hybas_lev02_bounds_coords[2][1] - hybas_lev02_bounds_coords[0][1])/4
    lat_upper_quarters = hybas_lev02_bounds_coords[2][1] - (hybas_lev02_bounds_coords[2][1] - hybas_lev02_bounds_coords[0][1])/4
    central_lat = (hybas_lev02_bounds_coords[0][1] + hybas_lev02_bounds_coords[2][1])/2
    central_lon = hybas_lev02_bounds_coords[0][0] + (hybas_lev02_bounds_coords[2][0] - hybas_lev02_bounds_coords[0][0])/2
    print(f'lat_lower_quarters: {lat_lower_quarters}, lat_upper_quarters: {lat_upper_quarters}, central_lat: {central_lat}, central_lon: {central_lon}')
    file_path = f'./projection_wkt/{MODE}_{PCS_SCALE}m/{hybas_lev02_id}.wkt'
    output_dir = os.path.dirname(file_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    lambert_conformal_conic(lat_lower_quarters, lat_upper_quarters, central_lon, central_lat, file_path, gcs_scale=GCS_SCALE, pcs_scale=PCS_SCALE)
    
    print(f"Hybas Lev02 ID: {hybas_lev02_id} projection WKT file saved to {file_path}")

In [ ]:
#test if exported wkt files are correct
from osgeo import osr

def read_wkt_and_create_srs(file_path):
    # Step 1: Read the WKT string from the file
    with open(file_path, 'r') as file:
        wkt_string = file.read()
    
    # Step 2: Create an osr.SpatialReference() object from the WKT string
    srs = osr.SpatialReference()
    srs_error = srs.ImportFromWkt(wkt_string)
    
    # Step 3: Test if the SRS object is valid
    if srs_error == 0:  # Returns 0 if the import was successful
        print("The WKT string is valid and the SRS object has been created successfully.")
        print(srs.ExportToWkt())  # Optionally print the SRS WKT to verify
    else:
        print("Error: The WKT string is not valid.")

# Example usage
file_path = './Lambert_Azimuthal_Equal_Area_30m/6020014330.wkt'  # Specify the path to your WKT file
read_wkt_and_create_srs(file_path)

In [ ]:
x = [[[30.52083438328381, -18.188066737266546], [54.538119010327335, -18.188066737266546], [54.538119010327335, 30.137498886324558], [30.52083438328381, 30.137498886324558], [30.52083438328381, -18.188066737266546]]]
x[0]